In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd -q ..

In [3]:
import lcdb
import json
import lcpfn 
import torch as th
from functools import partial
import numpy as np
from matplotlib import pyplot as pl

In [8]:
# path = "C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\Year 1\\Q3\\Deep Learning\\deep-learning-project\\notebooks\data.json"

def readDatasetJson(path):
    f = open(path)

    dataset = json.load(f)
    # print(len(dataset))
    
    x = []
    y = []

    for v in dataset.values():
        x.append(v[0])
        y.append(v[1])
        
    x_true = th.Tensor(x)
    y_true = th.Tensor(y)
    
    return x_true, y_true

In [9]:
import os
def find_file(root_dir, target_filename):
    for root, dirs, files in os.walk(root_dir):
        if target_filename in files:
            return os.path.join(root, target_filename)
    
    # If the loop completes without returning, the file was not found
    return None

def find_data():
    return find_file(os.getcwd(), 'data.json')

In [10]:
def get_data():
    return readDatasetJson(find_data())

x, y = readDatasetJson(find_data())
perm = th.randperm(x.size(0))
x = x[perm]
y = y[perm]

x_train = x[:x.shape[0]//2] 
x_test = x[x.shape[0]//2:]

y_train = y[:y.shape[0]//2] 
y_test = y[y.shape[0]//2:]

batch_size = x_train.shape[0]

def getBatch(training_set_x, training_set_y):
    return training_set_x, training_set_y

def getTrainBatchFunc():
    return partial(getBatch, x_train, y_train)

def getTestBatchFunc():
    return partial(getBatch, x_test, y_test)


In [11]:
# function producing batches for PFN training
def get_batch(
    batch_size,
    seq_len,
    num_features,
    device="cpu",
    noisy_target=True,
    **_,
):
    assert num_features == 1

    #x_data, y_data = get_data()
    func = getTrainBatchFunc()
    x_data, y_data = func()
    
    x_data = x_data[:batch_size, :]
    y_data = y_data[:batch_size, :]

    y_data_noisy = y_data.clone()

    # print(x_data.shape)
    # print(y_data.shape)

    x_data = x_data.view((num_features, batch_size, seq_len)).transpose(2, 0).to(device)
    y_data = y_data.transpose(1, 0).to(device)
    y_data_noisy = y_data.clone()




    # x = np.arange(1, seq_len + 1)
    # y_target = np.empty((batch_size, seq_len), dtype=float)
    # y_noisy = np.empty((batch_size, seq_len), dtype=float)
    # x_data, y_data = get_data() # uses numpy rng
    
    # for i in range(batch_size):    
    #     if noisy_target:
    #         y_noisy[i] = y_data[i]
    #         y_target[i] = y_data[i]
    #     else:
    #         y_target[i], y_noisy[i] = y_data[i]
    # # turn numpy arrays into correctly shaped torch tensors & move them to device
    # x = (
    #     th.Tensor(x_data[0])
    #     .repeat((num_features, batch_size, 1))
    #     .transpose(2, 0)
    #     .to(device)
    # )
    # y_target = th.from_numpy(y_target).transpose(1, 0).to(device)
    # y_noisy = th.from_numpy(y_noisy).transpose(1, 0).to(device)

    # # changes
    # x = x.float()
    # y_target = y_target.float()
    # y_noisy = y_noisy.float()
    
    # return x, y_noisy, y_target

    return x_data, y_data, y_data_noisy

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=256,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1651,
                         epochs=20)
th.save(result[2].state_dict(), 'model_1.txt')

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=128,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1651,
                         epochs=20)
th.save(result[2].state_dict(), 'model_2.txt')

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=64,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1651,
                         epochs=20)
th.save(result[2].state_dict(), 'model_3.txt')